In [ ]:
%matplotlib inline

import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from cselect import color as cs
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../samples')
from mvpose.data import epfl_campus
from time import time

root = Settings['data_root']
root = join(root, 'pak')
tmp = Settings['tmp']

import mvpose.data.kth_football2 as kth
from mvpose import pose
from mvpose.settings import get_settings
from paf_loader import Loader
from mvpose.evaluation import pcp

# =====================================
params = get_settings(scale_to_mm=1000,
                      hm_detection_threshold=0.05,
                      pp_conflict_overlap=0.4,
                      ms_radius=60,
                      ms_between_distance=60)
loader = Loader()
# =====================================

import mvpose.data.skeleton_augmentation as ska
model_path = '../data/model_poseprediction.h5'
gen = ska.LimbGenerator(model_path, params.scale_to_mm)

    
def generate_pcp_score(frame):
    global Settings, params
    alpha = 0.5
    
    Im, Y, Calib = epfl_campus.get(root, frame)
    heatmaps, pafs = loader.load_confidence_map_and_paf(
        'campus', Im, frame, dir=Settings['tmp'])
    detections = pose.estimate(Calib, heatmaps, pafs, 
                               settings=params, debug=False)
    #detections = gen.apply(detections)
    Humans = kth.transform3d_from_mscoco(detections)
    
    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    GTIDs = []
  
    for gtid, gt in enumerate(Y):
        if gt is None:
            continue
        
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        GTIDs.append(gtid)
    
    del heatmaps
    del pafs
    del detections
    del Im
    del Y
    del Calib
    
    return L_Arms, U_Arms, L_Legs, U_Legs, GTIDs

# =====================================

valid_frames = list(range(350, 470)) + list(range(650, 750))
PER_GTID = {}
FRAMES = []
AVG = []

for frame in valid_frames:
    _start = time()
    L_Arms, U_Arms, L_Legs, U_Legs, GTIDs =\
        generate_pcp_score(frame)
    _end = time()
    print('frame ' + str(frame) + ', elapsed:', _end - _start)
    if len(L_Arms) > 0:
        for gtid, larms, uarms, llegs, ulegs in zip(
            GTIDs, L_Arms, U_Arms, L_Legs, U_Legs
        ):
            if not gtid in PER_GTID:
                PER_GTID[gtid] = {
                    'larms': [],
                    'uarms': [],
                    'llegs': [],
                    'ulegs': [],
                    'frame': []
                }
            PER_GTID[gtid]['larms'].append(larms)
            PER_GTID[gtid]['uarms'].append(uarms)
            PER_GTID[gtid]['llegs'].append(llegs)
            PER_GTID[gtid]['ulegs'].append(ulegs)
            PER_GTID[gtid]['frame'].append(frame)
            
            FRAMES.append(frame)
            AVG.append((larms + uarms + llegs + ulegs)/4)
            
            

/home/tanke/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


frame 350, elapsed: 1.1801612377166748
frame 351, elapsed: 0.29469966888427734
frame 352, elapsed: 1.0901446342468262
frame 353, elapsed: 0.29528236389160156
frame 354, elapsed: 1.1435565948486328
frame 355, elapsed: 0.30182504653930664
frame 356, elapsed: 1.1156673431396484
frame 357, elapsed: 0.2993483543395996
frame 358, elapsed: 1.0654723644256592
frame 359, elapsed: 0.3134734630584717
frame 360, elapsed: 1.1396675109863281
frame 361, elapsed: 0.32541418075561523
frame 362, elapsed: 1.1222097873687744
frame 363, elapsed: 0.2993800640106201
frame 364, elapsed: 1.1162073612213135
frame 365, elapsed: 1.1149187088012695
frame 366, elapsed: 1.0719497203826904
frame 367, elapsed: 0.2948620319366455
frame 368, elapsed: 1.1306686401367188
frame 369, elapsed: 0.29468488693237305
frame 370, elapsed: 1.1146154403686523
frame 371, elapsed: 0.2870042324066162
frame 372, elapsed: 1.1097893714904785
frame 373, elapsed: 0.2745816707611084
frame 374, elapsed: 1.0650770664215088
frame 375, elapsed: 

In [ ]:
total_avg = []
for key, values in PER_GTID.items():
    print('actor ', key)
    print('\tuarms:', np.mean(values['uarms']))
    print('\tlarms:', np.mean(values['larms']))
    print('\tulegs:', np.mean(values['ulegs']))
    print('\tllegs:', np.mean(values['llegs']))
    avg = np.mean([
        np.mean(values['uarms']),
        np.mean(values['larms']),
        np.mean(values['ulegs']),
        np.mean(values['llegs'])
    ])
    total_avg.append(avg)
    print('\tavg:  ', avg)
print('\navg*:  ', np.mean(total_avg))

In [ ]:
# =====================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.4,
#                       ms_radius=60,
#                       ms_between_distance=60)
# MIT PC
# actor  1
# 	uarms: 0.9893617021276596
# 	larms: 0.5851063829787234
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8929521276595744
# actor  2
# 	uarms: 0.9779411764705882
# 	larms: 0.8051470588235294
# 	ulegs: 0.9889705882352942
# 	llegs: 0.9889705882352942
# 	avg:   0.9402573529411766
# actor  0
# 	uarms: 0.8229166666666666
# 	larms: 0.59375
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.8541666666666666

# avg*:   0.8957920490891392


# =====================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.4,
#                       ms_radius=90,
#                       ms_between_distance=70)
# MIT PC
# actor  1
# 	uarms: 0.9813829787234043
# 	larms: 0.5904255319148937
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8922872340425532
# actor  2
# 	uarms: 0.9742647058823529
# 	larms: 0.7977941176470589
# 	ulegs: 0.9816176470588235
# 	llegs: 0.9816176470588235
# 	avg:   0.9338235294117646
# actor  0
# 	uarms: 0.7916666666666666
# 	larms: 0.59375
# 	ulegs: 0.9791666666666666
# 	llegs: 0.9791666666666666
# 	avg:   0.8359374999999999

# avg*:   0.8873494211514393

# =====================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.4,
#                       ms_radius=40,
#                       ms_between_distance=70)
# MIT PC
# actor  1
# 	uarms: 0.9946808510638298
# 	larms: 0.5877659574468085
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8949468085106382
# actor  2
# 	uarms: 0.9742647058823529
# 	larms: 0.7977941176470589
# 	ulegs: 0.9889705882352942
# 	llegs: 0.9889705882352942
# 	avg:   0.9375
# actor  0
# 	uarms: 0.8229166666666666
# 	larms: 0.59375
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.8541666666666666

# avg*:   0.8955378250591016

# =====================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.4,
#                       ms_radius=70,
#                       ms_between_distance=40)
# # MIT PC
# actor  1
# 	uarms: 0.9893617021276596
# 	larms: 0.5851063829787234
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8929521276595744
# actor  2
# 	uarms: 0.9742647058823529
# 	larms: 0.8014705882352942
# 	ulegs: 0.9889705882352942
# 	llegs: 0.9889705882352942
# 	avg:   0.9384191176470589
# actor  0
# 	uarms: 0.8229166666666666
# 	larms: 0.6041666666666666
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.8567708333333333

# avg*:   0.8960473595466555


# =============================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.4,
#                       ms_radius=90,
#                       ms_between_distance=40)
# MIT PC
# actor  1
# 	uarms: 0.9813829787234043
# 	larms: 0.5877659574468085
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8916223404255319
# actor  2
# 	uarms: 0.9742647058823529
# 	larms: 0.7977941176470589
# 	ulegs: 0.9816176470588235
# 	llegs: 0.9816176470588235
# 	avg:   0.9338235294117646
# actor  0
# 	uarms: 0.7916666666666666
# 	larms: 0.59375
# 	ulegs: 0.9791666666666666
# 	llegs: 0.9791666666666666
# 	avg:   0.8359374999999999

# avg*:   0.8871277899457656

# =============================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.4,
#                       ms_radius=40,
#                       ms_between_distance=90)

# MIT PC
# actor  1
# 	uarms: 0.9813829787234043
# 	larms: 0.5851063829787234
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8909574468085106
# actor  2
# 	uarms: 0.9742647058823529
# 	larms: 0.7941176470588235
# 	ulegs: 0.9816176470588235
# 	llegs: 0.9816176470588235
# 	avg:   0.9329044117647058
# actor  0
# 	uarms: 0.8125
# 	larms: 0.5833333333333334
# 	ulegs: 0.9791666666666666
# 	llegs: 0.9791666666666666
# 	avg:   0.8385416666666666

# avg*:   0.8874678417466276
    
# ==============================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.4,
#                       ms_radius=80,
#                       ms_between_distance=80)
# MIT PC
# actor  1
# 	uarms: 0.9813829787234043
# 	larms: 0.5824468085106383
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8902925531914894
# actor  2
# 	uarms: 0.9742647058823529
# 	larms: 0.8088235294117647
# 	ulegs: 0.9889705882352942
# 	llegs: 0.9889705882352942
# 	avg:   0.9402573529411766
# actor  0
# 	uarms: 0.8229166666666666
# 	larms: 0.6145833333333334
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.859375

# avg*:   0.8966416353775554

# ==============================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.4,
#                       ms_radius=70,
#                       ms_between_distance=70)
# MIT PC
#actor  1
# 	uarms: 0.9893617021276596
# 	larms: 0.5851063829787234
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8929521276595744
# actor  2
# 	uarms: 0.9742647058823529
# 	larms: 0.8014705882352942
# 	ulegs: 0.9889705882352942
# 	llegs: 0.9889705882352942
# 	avg:   0.9384191176470589
# actor  0
# 	uarms: 0.8229166666666666
# 	larms: 0.59375
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.8541666666666666

# avg*:   0.8951793039911

# =============================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.4,
#                       ms_radius=30,
#                       ms_between_distance=30)

# MIT PC
# actor  1
# 	uarms: 0.9920212765957447
# 	larms: 0.5824468085106383
# 	ulegs: 0.9946808510638298
# 	llegs: 0.9893617021276596
# 	avg:   0.8896276595744681
# actor  2
# 	uarms: 0.9595588235294118
# 	larms: 0.7830882352941176
# 	ulegs: 0.9852941176470589
# 	llegs: 0.9852941176470589
# 	avg:   0.9283088235294118
# actor  0
# 	uarms: 0.8125
# 	larms: 0.6041666666666666
# 	ulegs: 0.9791666666666666
# 	llegs: 0.9791666666666666
# 	avg:   0.8437499999999999

# avg*:   0.8872288277012933

# ==============================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.6,
#                       ms_radius=50,
#                       ms_between_distance=50)
# OHNE PC
# actor  1
# 	uarms: 0.9787234042553191
# 	larms: 0.5797872340425532
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8889627659574468
# actor  2
# 	uarms: 0.9595588235294118
# 	larms: 0.7389705882352942
# 	ulegs: 0.9926470588235294
# 	llegs: 0.9926470588235294
# 	avg:   0.9209558823529412
# actor  0
# 	uarms: 0.8229166666666666
# 	larms: 0.5729166666666666
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.8489583333333333

# avg*:   0.8862923272145737

# MIT PC
# actor  1
# 	uarms: 0.9893617021276596
# 	larms: 0.5824468085106383
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8922872340425532
# actor  2
# 	uarms: 0.9669117647058824
# 	larms: 0.7941176470588235
# 	ulegs: 0.9926470588235294
# 	llegs: 0.9926470588235294
# 	avg:   0.9365808823529412
# actor  0
# 	uarms: 0.8333333333333334
# 	larms: 0.59375
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.8567708333333334

# avg*:   0.8952129832429426


# ==============================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       pp_conflict_overlap=0.8,
#                       ms_radius=50,
#                       ms_between_distance=50)
# actor  1
# 	uarms: 0.9680851063829787
# 	larms: 0.574468085106383
# 	ulegs: 0.9893617021276596
# 	llegs: 0.9867021276595744
# 	avg:   0.8796542553191489
# actor  2
# 	uarms: 0.9595588235294118
# 	larms: 0.7389705882352942
# 	ulegs: 0.9926470588235294
# 	llegs: 0.9926470588235294
# 	avg:   0.9209558823529412
# actor  0
# 	uarms: 0.8020833333333334
# 	larms: 0.5520833333333334
# 	ulegs: 0.9791666666666666
# 	llegs: 0.9791666666666666
# 	avg:   0.828125

# avg*:   0.8762450458906966


# ==============================================
# params = get_settings(scale_to_mm=1000,
#                       hm_detection_threshold=0.05,
#                       ms_radius=50,
#                       ms_between_distance=50)

# OHNE PC
# actor  1
# 	uarms: 0.9787234042553191
# 	larms: 0.5797872340425532
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8889627659574468
# actor  2
# 	uarms: 0.9595588235294118
# 	larms: 0.7389705882352942
# 	ulegs: 0.9926470588235294
# 	llegs: 0.9926470588235294
# 	avg:   0.9209558823529412
# actor  0
# 	uarms: 0.8229166666666666
# 	larms: 0.5729166666666666
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.8489583333333333

# avg*:   0.8862923272145737


# ==============================================
# params = get_settings(scale_to_mm=1000,
#                       ms_radius=50,
#                       ms_between_distance=50)


# OHNE PC
# actor  1
# 	uarms: 0.9840425531914894
# 	larms: 0.5930851063829787
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8936170212765957
# actor  2
# 	uarms: 0.9117647058823529
# 	larms: 0.6544117647058824
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.8915441176470589
# actor  0
# 	uarms: 0.75
# 	larms: 0.5416666666666666
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.8229166666666666

# avg*:   0.8693592685301071

# MIT PC
# actor  1
# 	uarms: 0.9973404255319149
# 	larms: 0.601063829787234
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.898936170212766
# actor  2
# 	uarms: 0.9779411764705882
# 	larms: 0.8125
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.9476102941176471
# actor  0
# 	uarms: 0.7708333333333334
# 	larms: 0.5416666666666666
# 	ulegs: 1.0
# 	llegs: 1.0
# 	avg:   0.828125

# avg*:   0.8915571547768044

# ==============================================

# standard settings
# MIT PC
# actor  1
# 	uarms: 0.9920212765957447
# 	larms: 0.6090425531914894
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8996010638297872
# actor  2
# 	uarms: 0.9779411764705882
# 	larms: 0.8272058823529411
# 	ulegs: 0.9742647058823529
# 	llegs: 0.9742647058823529
# 	avg:   0.9384191176470588
# actor  0
# 	uarms: 0.71875
# 	larms: 0.5104166666666666
# 	ulegs: 0.9375
# 	llegs: 0.9375
# 	avg:   0.7760416666666666

# avg*:   0.871353949381171

# OHNE PC
# actor  1
# 	uarms: 0.9813829787234043
# 	larms: 0.601063829787234
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8949468085106383
# actor  2
# 	uarms: 0.9080882352941176
# 	larms: 0.6617647058823529
# 	ulegs: 0.9742647058823529
# 	llegs: 0.9742647058823529
# 	avg:   0.879595588235294
# actor  0
# 	uarms: 0.6770833333333334
# 	larms: 0.5104166666666666
# 	ulegs: 0.9375
# 	llegs: 0.9375
# 	avg:   0.765625

# avg*:   0.8467224655819775

In [ ]:
idx = 194

fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.plot(range(len(AVG)), AVG)
ax.set_ylim([0, 1.1])

ax.plot([idx, idx], [0, 1.5], alpha=0.5)
 
print("avg:\t", AVG[idx])
print('frame:\t', FRAMES[idx])
FRAME = FRAMES[idx]

plt.show()

In [ ]:
COLORS = ['red', 'green', 'blue', 'yellow']

# ==================================
from mvpose import pose
from mvpose.settings import get_settings
import paf_loader
from mvpose.plot.limbs import draw_mscoco_human

Im, Y, Calib = epfl_campus.get(root, FRAME)

heatmaps, pafs = paf_loader.load_confidence_map_and_paf('campus', 
                                                        Im, 
                                                        FRAME, 
                                                        dir=tmp)


def get_pcp(detections, Y):
    alpha = 0.5
    Humans = kth.transform3d_from_mscoco(detections)

    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    GTIDs = []

    for gtid, gt in enumerate(Y):
        if gt is None:
            continue
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        GTIDs.append(gtid)
    return L_Arms, U_Arms, L_Legs, U_Legs, GTIDs



#param = get_settings(scale_to_mm=1000)
param = get_settings(
    scale_to_mm=1000,
    ms_radius=50,
    ms_between_distance=50,
    hm_detection_threshold=0.05
)

Debug, detections = pose.estimate(Calib, heatmaps, pafs,
                                  settings=param,
                                  debug=True)

print('\n# detections', len(detections))

#detections = gen.apply(detections)


L_Arms, U_Arms, L_Legs, U_Legs, GTIDs = get_pcp(detections, Y)

print('------')
for larms, uarms, llegs, ulegs, gtid in zip(L_Arms, U_Arms, L_Legs, U_Legs, GTIDs):
    print('gtid:', gtid)
    print('\tlarms:', larms)
    print('\tuarms:', uarms)
    print('\tllegs:', llegs)
    print('\tulegs:', ulegs)
    

avg = np.mean(L_Arms) + np.mean(U_Arms) + np.mean(L_Legs) + np.mean(U_Legs)
print('avg:', avg/4)

Humans = detections

fig = plt.figure(figsize=(16,12))

colors = ['blue', 'red', 'green']

for idx, cam in enumerate(Calib):
    ax = fig.add_subplot(2, 2,1+idx)
    ax.axis('off')
    im = Im[idx]
    h,w,_ = im.shape
    ax.set_xlim([0, w])
    ax.set_ylim([h, 0])
    ax.imshow(im, alpha=0.6)
    
    for pid, human in enumerate(Humans):
        draw_mscoco_human(ax, human, cam, color=colors[pid])
        
plt.tight_layout()
plt.show()